In [1]:
import os, sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchtext 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm, model_selection, preprocessing, naive_bayes

In [2]:
dataFile = '../Data/us_equities_news_dataset_clean.csv'

In [3]:
dataFrame = pd.read_csv(dataFile)
dataFrame.head(1)

,id,ticker,title,category,content,release_date,provider,url,article_id
0,221515,NIO,shares chinese electric car maker nio flying h...,news,happeningshares chinese electric car maker nio...,2020-01-15,The Motley Fool,https://invst.ly/pigqi,2060327


In [4]:
#cleanDataFrame = pd.DataFrame()
stockDF = pd.DataFrame(dataFrame.loc[dataFrame['ticker'] == 'AAPL'])
#appleDF = 
stockDF.rename(columns={'release_date':'Date','content':'Text'}, inplace=True)
#sorted(set(appleDF['release_date']))
stockDF.head()

,id,ticker,title,category,Text,Date,provider,url,article_id
49183,270698,AAPL,jpmorgan cautious ahead apple earnings,news,jpmorgan lifts apple aapl 2 9 target 296 300 a...,2020-01-28,Seeking Alpha,https://invst.ly/pnjv8,2068762
49184,270699,AAPL,faang fall get wall street love,news,kim khan investing com faang stocks predictabl...,2020-01-28,Investing.com,https://www.investing.com/news/stock-market-ne...,2068765
49185,270700,AAPL,wall street tumbles virus fuels economic worry,news,chuck mikolajczak new york reuters stocks suff...,2020-01-28,Reuters,https://www.investing.com/news/stock-market-ne...,2068311
49186,270701,AAPL,earnings watch apple amd take earnings stage a...,news,two best performing tech stocks 2019 set repor...,2020-01-28,MarketWatch,https://invst.ly/pnlbs,2068906
49187,270702,AAPL,day ahead top 3 things watch jan 28,news,yasin ebrahim kim khan 1 apple readies earning...,2020-01-28,Investing.com,https://www.investing.com/news/stock-market-ne...,2068907


In [5]:
def dateConvert(input):
    input = input.split('/')
    return input[2]+'-'+input[0]+'-'+input[1]

In [6]:
priceFile = '../Data/HistoricalData_AAPL.csv'
priceDF = pd.read_csv(priceFile, converters={'Date': dateConvert})
priceDF['Label'] = (priceDF['Open'] > priceDF['Close/Last']) * 1
priceDF.head()

,Date,Close/Last,Volume,Open,High,Low,Label
0,2022-12-05,$146.63,68826440,$147.77,$150.9199,$145.77,1
1,2022-12-02,$147.81,65447450,$145.96,$148,$145.65,0
2,2022-12-01,$148.31,71250420,$148.21,$149.13,$146.61,0
3,2022-11-30,$148.03,111380900,$141.395,$148.72,$140.55,0
4,2022-11-29,$141.17,83763800,$144.29,$144.81,$140.355,1


In [7]:
SHUFFLE_SEED = 12345
combinedDF = pd.DataFrame()
combinedDF['Date'] = priceDF['Date']
combinedDF['Label'] = priceDF['Label']
combinedDF = combinedDF.merge(stockDF[['Text','Date']], on='Date')
combinedDF = combinedDF.sample(frac=1, random_state=SHUFFLE_SEED)

In [8]:
N = len(combinedDF)
BATCH_SIZE = 16
TRAIN_PERCENT = 0.8
VALID_PERCENT = 0.2

trainSize = int(N * TRAIN_PERCENT)
#validSize = int(trainSize * VALID_PERCENT)

train_newsDataFrame = combinedDF.iloc[:trainSize]
#valid_df = combinedDF.iloc[trainSize-validSize:trainSize]
test_newsDataFrame = combinedDF.iloc[trainSize: N]

In [9]:

train_newsDataFrame.head()

,Date,Label,Text
16170,2015-03-11,1,netflix nasdaq nflx continues expand internati...
14193,2016-07-12,0,immediate releasechicago il july 13 2016 stock...
6997,2018-05-07,1,henry schein inc nasdaq hsic expected report f...
15593,2015-08-11,1,going today trade stats leaning negative senti...
5916,2018-09-12,1,tuesday p 500 slipped open china filed petitio...


In [10]:

train_data = []
train_labels = []
for row in train_newsDataFrame.values:
    train_data.append(row[2])
    train_labels.append(row[1])

test_data = [] 
test_labels = [] 
for row in test_newsDataFrame.values:
    test_data.append(row[2]) 
    test_labels.append(row[1])

In [11]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(test_data)

In [12]:
train_vectors[0]

<1x24333 sparse matrix of type '<class 'numpy.float64'>'
	with 262 stored elements in Compressed Sparse Row format>

In [13]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, train_labels) 
prediction = model.predict(test_vectors)
print (classification_report(test_labels, prediction))

              precision    recall  f1-score   support

           0       0.57      0.69      0.63      1895
           1       0.52      0.40      0.45      1603

    accuracy                           0.56      3498
   macro avg       0.55      0.54      0.54      3498
weighted avg       0.55      0.56      0.55      3498



In [14]:
trainDF = pd.DataFrame()
trainDF['text'] = test_data
trainDF['label'] = test_labels

In [15]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

TF-IDF

In [16]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

Count Vectors

In [17]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [18]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy_score(predictions, valid_y)

In [19]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.5394285714285715
NB, WordLevel TF-IDF:  0.5428571428571428
NB, N-Gram Vectors:  0.5097142857142857
NB, CharLevel Vectors:  0.5622857142857143


In [20]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.5337142857142857
